In [105]:
# Lets get into modeling. First we are going to do our standard imports to acquire, prepare, and split our data 

# First im going to acquire my data :
import wrangle as w
import explore as e
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 

# imports for warnings 
import warnings
warnings.filterwarnings("ignore")

# import our models we want to test 
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
# Here i want to acquire my data and store it in a variable named df (short for dataframe)
df = w.prepare()
train, validate, test = w.split(w.prepare())
X_train, y_train, X_validate, y_validate, X_test, y_test = e.X_and_y_split(train, validate, test)
#Here we are checking which has more churned or not churn to set up our baseline


print((train['churn'] == 1).sum())
print((train['churn'] == 0).sum())

1064
2879


## Do we have imported our data. We have split it in to train, validate, and test.  Now I want to set some baseline metrics

# Baseline Model

In [3]:
# First we set a new dataframe specifically to the X train values 
base_line_df = X_train.copy()
base_line_df['prediction'] = 0 

In [4]:
# For the purpose of our confusion matrixs we are defining the following variables:
y_true = y_train
y_predicted = base_line_df['prediction']

In [5]:
cm = confusion_matrix(y_true, y_predicted)

cm

array([[2879,    0],
       [1064,    0]])

In [6]:
base_line_df['actual'] = y_train

In [8]:
baseline_accuracy = (base_line_df.prediction == base_line_df.actual).mean()

In [9]:
print(f'baseline accuracy: {baseline_accuracy:.2%}')

baseline accuracy: 73.02%


In [79]:
y_true_baseline = y_train
y_pred_baseline = base_line_df.prediction
print(classification_report(y_true_baseline, y_pred_baseline))

              precision    recall  f1-score   support

           0       0.73      1.00      0.84      2879
           1       0.00      0.00      0.00      1064

    accuracy                           0.73      3943
   macro avg       0.37      0.50      0.42      3943
weighted avg       0.53      0.73      0.62      3943



# Desicion Tree Model 

In [10]:
clf = DecisionTreeClassifier(max_depth=10, random_state=123)
clf = clf.fit(X_train,y_train)
y_pred = clf.predict(X_train)
decision_tree_model = X_train.copy()
decision_tree_model['predicted'] = y_pred
decision_tree_model['actual'] = y_train
decision_tree_model['baseline_pred'] = 0

In [ ]:
decision_tree_model.head()

,contract_type_one_year,contract_type_two_year,monthly_charges,total_charges,predicted,actual
4604,0,0,88.65,2683.20,0,0
5566,0,0,52.10,670.65,0,0
6204,1,0,65.40,2498.40,0,1
5837,0,0,87.25,3941.70,0,1
1276,0,1,90.60,5817.45,0,0


In [ ]:
decision_tree_model.predicted.value_counts()

0    2917
1    1026
Name: predicted, dtype: int64

    Lets takes some metrics on our models 

In [19]:
model_accuracy = (decision_tree_model.predicted == decision_tree_model.actual).mean()
baseline_accuracy = (decision_tree_model.baseline_pred == decision_tree_model.actual).mean()

print(f'   model accuracy: {model_accuracy:.2%}')
print(f'baseline accuracy: {baseline_accuracy:.2%}')
print(f' % of model - baseline accuracy:{(model_accuracy - baseline_accuracy )* 100}')

   model accuracy: 85.09%
baseline accuracy: 73.02%
 % of model - baseline accuracy:12.072026375855948


In [29]:
y_true = y_train
y_pred = decision_tree_model.predicted
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.89      0.90      0.90      2879
           1       0.73      0.71      0.72      1064

    accuracy                           0.85      3943
   macro avg       0.81      0.81      0.81      3943
weighted avg       0.85      0.85      0.85      3943



# KNN Model 

In [41]:
knn_df = X_train.copy()
knn = KNeighborsClassifier(n_neighbors=10, weights='distance')

knn.fit(X_train, y_train)

y_pred_knn = knn.predict(X_train)

y_pred_proba = knn.predict_proba(X_train)
print(classification_report(y_train, y_pred_knn))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      2879
           1       0.99      0.98      0.98      1064

    accuracy                           0.99      3943
   macro avg       0.99      0.99      0.99      3943
weighted avg       0.99      0.99      0.99      3943



In [81]:
knn_df = X_train.copy()
knn = KNeighborsClassifier(n_neighbors=10, weights='distance')

knn.fit(X_train, y_train)

y_pred_knn = knn.predict(X_train)

y_pred_proba = knn.predict_proba(X_train)
print(classification_report(y_train, y_pred_knn))
knn_class_report = classification_report(y_train, y_pred_knn)

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      2879
           1       0.99      0.98      0.98      1064

    accuracy                           0.99      3943
   macro avg       0.99      0.99      0.99      3943
weighted avg       0.99      0.99      0.99      3943



In [ ]:
knn_df['actual'] = y_train
knn_df['predicted'] = y_pred_knn

model_accuracy = (knn_df.predicted == decision_tree_model.actual).mean()
baseline_accuracy = (decision_tree_model.baseline_pred == decision_tree_model.actual).mean()


print(f'   model accuracy: {model_accuracy:.2%}')
print(f'baseline accuracy: {baseline_accuracy:.2%}')
print(f' % of model - baseline accuracy:{(model_accuracy - baseline_accuracy )* 100}')

   model accuracy: 99.19%
baseline accuracy: 73.02%
 % of model - baseline accuracy:26.17296474765407


    Here we are testing the validate on our model and see how it does 

In [86]:
y_pred_knn_val = knn.predict(X_validate)

y_pred_proba_val = knn.predict_proba(X_validate)
print(classification_report(y_validate, y_pred_knn_val))

              precision    recall  f1-score   support

           0       0.83      0.89      0.86      1263
           1       0.57      0.45      0.51       428

    accuracy                           0.78      1691
   macro avg       0.70      0.67      0.68      1691
weighted avg       0.76      0.78      0.77      1691



In [92]:
knn_df_val = X_validate.copy()
knn_df_val['actual'] = y_validate
knn_df_val['predicted_validate'] = y_pred_knn_val

In [96]:
knn_df_val.head()

,contract_type_one_year,contract_type_two_year,monthly_charges,total_charges,actual,predicted_validate
5854,0,0,66.40,831.75,0,0
797,0,1,109.30,7337.55,0,0
3188,0,0,18.85,163.20,0,0
2254,0,0,51.10,1269.60,0,0
5732,0,0,19.75,19.75,0,0


In [97]:
base_line_df.head(1)

,contract_type_one_year,contract_type_two_year,monthly_charges,total_charges,prediction,actual
4604,0,0,88.65,2683.2,0,0


# Logistic Regression Model

In [74]:
# from sklearn.linear_model import LogisticRegression
logit = LogisticRegression(C=1, class_weight={0:1, 1:99}, random_state=123, intercept_scaling=1, solver='lbfgs')
logit.fit(X_train, y_train)
y_pred_logit = logit.predict(X_train)
logit_df = X_train.copy()
logit_df['actual'] = y_train
logit_df['predicted'] = y_pred_logit
print(classification_report(y_train, y_pred_logit))

              precision    recall  f1-score   support

           0       0.99      0.03      0.05      2879
           1       0.27      1.00      0.43      1064

    accuracy                           0.29      3943
   macro avg       0.63      0.51      0.24      3943
weighted avg       0.79      0.29      0.15      3943



In [76]:
# now for it set to c 
logit2 = LogisticRegression(C=.1, class_weight={0:1, 1:99}, random_state=123, intercept_scaling=1, solver='lbfgs')
logit2.fit(X_train, y_train)
y_pred2 = logit2.predict(X_train)
y_pred_proba2 = logit2.predict_proba(X_train)

print(classification_report(y_train, y_pred2))

              precision    recall  f1-score   support

           0       1.00      0.00      0.01      2879
           1       0.27      1.00      0.43      1064

    accuracy                           0.27      3943
   macro avg       0.64      0.50      0.22      3943
weighted avg       0.80      0.27      0.12      3943



In [ ]:
print('Coefficient: \n', logit.coef_)
print('Intercept: \n', logit.intercept_)


Coefficient: 
 [[-1.64947634e+00 -2.64109290e+00  3.50166042e-02 -2.33799206e-04]]
Intercept: 
 [2.34206615]


| Metric     | Explanation |
|------------|-------------|
| Precision  | The proportion of predicted positives that are actually positive. A high precision score indicates that the classifier is good at identifying true positives and has a low false positive rate. |
| Recall     | The proportion of actual positives that were correctly identified by the classifier. A high recall score indicates that the classifier is good at identifying all positive instances, and has a low false negative rate. |
| F1-score   | A weighted average of precision and recall, and provides a balanced measure of the classifier's performance. It combines precision and recall into a single score that ranges from 0 to 1, with 1 being the best possible score. |
| Support    | The number of samples in each class in the test set. |
| Accuracy   | The percentage of correct predictions over the total number of predictions made. In this case, the accuracy is 0.83, indicating the classifier correctly classified 83% of the samples in the test set. |
| Macro Avg  | The unweighted average of precision, recall, and F1-score across both classes. |
| Weighted Avg | The weighted average of precision, recall, and F1-score across both classes, weighted by the number of samples in each class. |


# Random Forest 

In [114]:
rf = RandomForestClassifier(bootstrap=True, 
                            class_weight=None, 
                            criterion='gini',
                            min_samples_leaf=3,
                            n_estimators=100,
                            max_depth=3, 
                            random_state=123)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_train)
random_forest = X_train.copy()
random_forest['actual'] = y_train
random_forest['predicted'] = y_pred_rf

In [118]:
print(classification_report(y_train, y_pred_rf))

              precision    recall  f1-score   support

           0       0.77      0.97      0.86      2879
           1       0.73      0.23      0.36      1064

    accuracy                           0.77      3943
   macro avg       0.75      0.60      0.61      3943
weighted avg       0.76      0.77      0.72      3943



In [119]:
print('Accuracy of random forest classifier on test set: {:.2f}'
     .format(rf.score(X_validate, y_validate)))

Accuracy of random forest classifier on test set: 0.79
